# Imports

In [13]:
import math
import pandas as pd
import pennylane as qml
import time

from matplotlib import pyplot as plt
from pennylane import numpy as np
from pennylane.templates import AmplitudeEmbedding, AngleEmbedding
from pennylane.templates.subroutines import ArbitraryUnitary
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Model Params

In [14]:
np.random.seed(42)
initial_params = np.random.random([105])

INITIALIZATION_METHOD = 'Angle'
BATCH_SIZE = 20
EPOCHS = 400

STEP_SIZE = 0.01
BETA_1 = 0.9
BETA_2 = 0.99
EPSILON = 0.00000001

TRAINING_SIZE = 0.78
VALIDATION_SIZE = 0.07
TEST_SIZE = 1-TRAINING_SIZE-VALIDATION_SIZE

initial_time = time.time()

# Fetch Dataset

In [15]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')

# Import dataset

In [16]:
examples = mnist.data
classes = mnist.target

x = []
y = []
for (example, label) in zip(examples, classes):
    if label in ["0", "1", "2", "3"]:
        x.append(example)
        y.append(-1)
    else:
        x.append(example)
        y.append(1)

In [17]:
x = np.array(x)
y = np.array(y)

# Normalize pixels values
x = x / 255

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE, shuffle=True)

In [18]:
validation_indexes = np.random.random_integers(len(X_train), size=(math.floor(len(X_train)*VALIDATION_SIZE),))
X_validation = [X_train[n] for n in validation_indexes]
y_validation = [y_train[n] for n in validation_indexes]

In [19]:
pca = PCA(n_components=8)
pca.fit(X_train)
X_train = pca.transform(X_train)
X_validation = pca.transform(X_validation)
X_test = pca.transform(X_test)

preprocessing_time = time.time()

# Circuit creation

In [20]:
device = qml.device("default.qubit", wires=8)

In [21]:
@qml.qnode(device)
def circuit(features, params):
    # Load state
    if INITIALIZATION_METHOD == 'Amplitude':
        AmplitudeEmbedding(features=features, wires=range(8), normalize=True, pad_with=0.)
    else:
        AngleEmbedding(features=features, wires=range(8), rotation='Y')

    # First layer
    ArbitraryUnitary(params[:15], wires=[0, 1])
    ArbitraryUnitary(params[15:30], wires=[2, 3])
    ArbitraryUnitary(params[30:45], wires=[4, 5])
    ArbitraryUnitary(params[45:60], wires=[6, 7])

    # Second layer
    ArbitraryUnitary(params[60:75], wires=[1, 2])
    ArbitraryUnitary(params[75:90], wires=[5, 6])

    # Third layer
    ArbitraryUnitary(params[90:105], wires=[2, 5])

    # Measurement
    return qml.expval(qml.PauliZ(5))

## Circuit example

In [22]:
features = X_train[0]
print(f"Inital parameters: {initial_params}\n")
print(f"Example features: {features}\n")
print(f"Expectation value: {circuit(features, initial_params)}\n")
print(circuit.draw())

Inital parameters: [0.37454012 0.95071431 0.73199394 0.59865848 0.15601864 0.15599452
 0.05808361 0.86617615 0.60111501 0.70807258 0.02058449 0.96990985
 0.83244264 0.21233911 0.18182497 0.18340451 0.30424224 0.52475643
 0.43194502 0.29122914 0.61185289 0.13949386 0.29214465 0.36636184
 0.45606998 0.78517596 0.19967378 0.51423444 0.59241457 0.04645041
 0.60754485 0.17052412 0.06505159 0.94888554 0.96563203 0.80839735
 0.30461377 0.09767211 0.68423303 0.44015249 0.12203823 0.49517691
 0.03438852 0.9093204  0.25877998 0.66252228 0.31171108 0.52006802
 0.54671028 0.18485446 0.96958463 0.77513282 0.93949894 0.89482735
 0.59789998 0.92187424 0.0884925  0.19598286 0.04522729 0.32533033
 0.38867729 0.27134903 0.82873751 0.35675333 0.28093451 0.54269608
 0.14092422 0.80219698 0.07455064 0.98688694 0.77224477 0.19871568
 0.00552212 0.81546143 0.70685734 0.72900717 0.77127035 0.07404465
 0.35846573 0.11586906 0.86310343 0.62329813 0.33089802 0.06355835
 0.31098232 0.32518332 0.72960618 0.6375574

# Accuracy test definition

In [23]:
def measure_accuracy(x, y, circuit_params):
    class_errors = 0

    for example, example_class in zip(x, y):
        predicted_value = circuit(example, circuit_params)

        if (example_class > 0 and predicted_value <= 0) or (example_class <= 0 and predicted_value > 0):
            class_errors += 1

    return 1 - (class_errors/len(y))

# Training

In [24]:
params = initial_params
opt = qml.AdamOptimizer(stepsize=STEP_SIZE, beta1=BETA_1, beta2=BETA_2, eps=EPSILON)
test_accuracies = []
best_validation_accuracy = 0.0
best_params = []

for i in range(len(X_train)):
    features = X_train[i]
    expected_value = y_train[i]

    def cost(circuit_params):
        value = circuit(features, circuit_params)
        return ((expected_value - value) ** 2)/len(X_train)

    params = opt.step(cost, params)

    if i % BATCH_SIZE == 0:
        print(f"epoch {i//BATCH_SIZE}")

    if i % (10*BATCH_SIZE) == 0:
        current_accuracy = measure_accuracy(X_validation, y_validation, params)
        test_accuracies.append(current_accuracy)
        print(f"accuracy: {current_accuracy}")

        if current_accuracy > best_validation_accuracy:
            print("best accuracy so far!")
            best_validation_accuracy = current_accuracy
            best_params = params

    if len(test_accuracies) == 30:
        print(f"test_accuracies: {test_accuracies}")

        if np.allclose(best_validation_accuracy, test_accuracies[0]):
            params = best_params
            break

        del test_accuracies[0]

epoch 0
accuracy: 0.4689075630252101
best accuracy so far!
epoch 1
epoch 2
epoch 3
epoch 4
epoch 5
epoch 6
epoch 7
epoch 8
epoch 9
epoch 10
accuracy: 0.6890756302521008
best accuracy so far!
epoch 11
epoch 12
epoch 13
epoch 14
epoch 15
epoch 16
epoch 17
epoch 18
epoch 19
epoch 20
accuracy: 0.7135654261704683
best accuracy so far!
epoch 21
epoch 22
epoch 23
epoch 24
epoch 25
epoch 26
epoch 27
epoch 28
epoch 29
epoch 30
accuracy: 0.7018007202881152
epoch 31
epoch 32
epoch 33
epoch 34
epoch 35
epoch 36
epoch 37
epoch 38
epoch 39
epoch 40
accuracy: 0.700360144057623
epoch 41
epoch 42
epoch 43
epoch 44
epoch 45
epoch 46
epoch 47
epoch 48
epoch 49
epoch 50
accuracy: 0.735654261704682
best accuracy so far!
epoch 51
epoch 52
epoch 53
epoch 54
epoch 55
epoch 56
epoch 57
epoch 58
epoch 59
epoch 60
accuracy: 0.7109243697478991
epoch 61
epoch 62
epoch 63
epoch 64
epoch 65
epoch 66
epoch 67
epoch 68
epoch 69
epoch 70
accuracy: 0.7195678271308523
epoch 71
epoch 72
epoch 73
epoch 74
epoch 75
epoch 76

epoch 342
epoch 343
epoch 344
epoch 345
epoch 346
epoch 347
epoch 348
epoch 349
epoch 350
accuracy: 0.7390156062424971
test_accuracies: [0.7109243697478991, 0.7195678271308523, 0.7070828331332533, 0.7284513805522208, 0.7190876350540216, 0.6914765906362546, 0.7205282112845138, 0.7186074429771909, 0.7217286914765906, 0.7046818727490997, 0.7296518607442977, 0.7368547418967587, 0.7433373349339736, 0.7402160864345738, 0.7090036014405763, 0.7330132052821128, 0.7282112845138056, 0.7546218487394958, 0.7198079231692678, 0.7639855942376951, 0.7471788715486194, 0.75078031212485, 0.7483793517406963, 0.7452581032412966, 0.765906362545018, 0.7378151260504202, 0.7450180072028811, 0.7459783913565425, 0.7531812725090036, 0.7390156062424971]
epoch 351
epoch 352
epoch 353
epoch 354
epoch 355
epoch 356
epoch 357
epoch 358
epoch 359
epoch 360
accuracy: 0.7375750300120048
test_accuracies: [0.7195678271308523, 0.7070828331332533, 0.7284513805522208, 0.7190876350540216, 0.6914765906362546, 0.7205282112845138,

epoch 453
epoch 454
epoch 455
epoch 456
epoch 457
epoch 458
epoch 459
epoch 460
accuracy: 0.7426170468187274
test_accuracies: [0.7368547418967587, 0.7433373349339736, 0.7402160864345738, 0.7090036014405763, 0.7330132052821128, 0.7282112845138056, 0.7546218487394958, 0.7198079231692678, 0.7639855942376951, 0.7471788715486194, 0.75078031212485, 0.7483793517406963, 0.7452581032412966, 0.765906362545018, 0.7378151260504202, 0.7450180072028811, 0.7459783913565425, 0.7531812725090036, 0.7390156062424971, 0.7375750300120048, 0.7286914765906363, 0.6981992797118848, 0.75078031212485, 0.7464585834333733, 0.7368547418967587, 0.7709483793517407, 0.7661464585834333, 0.763265306122449, 0.7387755102040816, 0.7426170468187274]
epoch 461
epoch 462
epoch 463
epoch 464
epoch 465
epoch 466
epoch 467
epoch 468
epoch 469
epoch 470
accuracy: 0.7169267707082834
test_accuracies: [0.7433373349339736, 0.7402160864345738, 0.7090036014405763, 0.7330132052821128, 0.7282112845138056, 0.7546218487394958, 0.7198079231

epoch 568
epoch 569
epoch 570
accuracy: 0.7490996398559424
test_accuracies: [0.7483793517406963, 0.7452581032412966, 0.765906362545018, 0.7378151260504202, 0.7450180072028811, 0.7459783913565425, 0.7531812725090036, 0.7390156062424971, 0.7375750300120048, 0.7286914765906363, 0.6981992797118848, 0.75078031212485, 0.7464585834333733, 0.7368547418967587, 0.7709483793517407, 0.7661464585834333, 0.763265306122449, 0.7387755102040816, 0.7426170468187274, 0.7169267707082834, 0.7647058823529411, 0.75078031212485, 0.7601440576230492, 0.7769507803121248, 0.7330132052821128, 0.7726290516206482, 0.767827130852341, 0.7594237695078031, 0.7457382953181273, 0.7490996398559424]
epoch 571
epoch 572
epoch 573
epoch 574
epoch 575
epoch 576
epoch 577
epoch 578
epoch 579
epoch 580
accuracy: 0.77046818727491
test_accuracies: [0.7452581032412966, 0.765906362545018, 0.7378151260504202, 0.7450180072028811, 0.7459783913565425, 0.7531812725090036, 0.7390156062424971, 0.7375750300120048, 0.7286914765906363, 0.6981

epoch 681
epoch 682
epoch 683
epoch 684
epoch 685
epoch 686
epoch 687
epoch 688
epoch 689
epoch 690
accuracy: 0.765906362545018
test_accuracies: [0.7464585834333733, 0.7368547418967587, 0.7709483793517407, 0.7661464585834333, 0.763265306122449, 0.7387755102040816, 0.7426170468187274, 0.7169267707082834, 0.7647058823529411, 0.75078031212485, 0.7601440576230492, 0.7769507803121248, 0.7330132052821128, 0.7726290516206482, 0.767827130852341, 0.7594237695078031, 0.7457382953181273, 0.7490996398559424, 0.77046818727491, 0.7654261704681873, 0.7589435774309724, 0.7649459783913566, 0.7726290516206482, 0.7483793517406963, 0.7490996398559424, 0.7642256902761104, 0.7503001200480193, 0.7368547418967587, 0.7466986794717887, 0.765906362545018]
epoch 691
epoch 692
epoch 693
epoch 694
epoch 695
epoch 696
epoch 697
epoch 698
epoch 699
epoch 700
accuracy: 0.7603841536614646
test_accuracies: [0.7368547418967587, 0.7709483793517407, 0.7661464585834333, 0.763265306122449, 0.7387755102040816, 0.7426170468187

epoch 797
epoch 798
epoch 799
epoch 800
accuracy: 0.7503001200480193
test_accuracies: [0.7769507803121248, 0.7330132052821128, 0.7726290516206482, 0.767827130852341, 0.7594237695078031, 0.7457382953181273, 0.7490996398559424, 0.77046818727491, 0.7654261704681873, 0.7589435774309724, 0.7649459783913566, 0.7726290516206482, 0.7483793517406963, 0.7490996398559424, 0.7642256902761104, 0.7503001200480193, 0.7368547418967587, 0.7466986794717887, 0.765906362545018, 0.7603841536614646, 0.7627851140456182, 0.7303721488595438, 0.7618247298919568, 0.7459783913565425, 0.7644657863145258, 0.7510204081632653, 0.751500600240096, 0.7599039615846339, 0.756062424969988, 0.7503001200480193]


In [25]:
print("Optimized rotation angles: {}".format(params))

training_time = time.time()

Optimized rotation angles: [-1.55617034e-01  1.21653127e-01  5.47921643e-01  3.21062753e-01
 -3.75450529e-02  4.99268700e-01 -1.36039505e+00  1.44549989e+00
  5.98251096e-01  8.49800987e-01 -4.23941621e-02  2.16715865e+00
  1.25550000e+00 -1.04369315e-01  4.11721098e-01 -5.28883991e-01
  1.42781329e-02 -9.06485777e-02 -2.38468328e-01  1.16852143e-01
  1.22327998e+00  4.92869536e-01  9.07414100e-01 -1.58510422e-01
  9.27790892e-01 -6.52156318e-02 -6.45050129e-02  2.57005799e-01
 -2.83979412e-01  4.64504127e-02  1.19867042e+00  9.00909818e-01
 -7.13975538e-01  9.55134248e-01  3.38571598e+00 -3.77117609e-01
 -1.21602608e+00  8.30031821e-01  1.16429325e+00  5.52875392e-01
  1.83355701e-01  1.41713769e+00  5.38278814e-02  6.87048074e-01
 -2.80117715e-01  5.91637454e-02  7.54161178e-01  3.26137551e-01
  1.16916822e+00  5.82013681e-01  1.42376270e+00  5.29827519e-01
  1.21513067e+00  4.00233480e-01  2.06763115e-01  4.85669294e-01
  7.49139882e-01 -4.44714839e-01  6.26980693e-01  3.25330331e-0

# Testing

In [26]:
accuracy = measure_accuracy(X_test, y_test, params)
print(accuracy)

test_time = time.time()

0.7676190476190476


In [27]:
print(f"pre-processing time: {preprocessing_time-initial_time}")
print(f"training time: {training_time - preprocessing_time}")
print(f"test time: {test_time - training_time}")
print(f"total time: {test_time - initial_time}")

pre-processing time: 67.53380346298218
training time: 49246.52304840088
test time: 1155.7286159992218
total time: 50469.78546786308
